<a href="https://colab.research.google.com/github/Sbarbagnem/AdvancedProject/blob/master/ProgettoAdvancedMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multi-label object classification

## Componenti del gruppo
* Carta Costantino
* Hamrani Hamza
* Ventura Samuele 793060

## Module Import


In [1]:
from google.colab import drive
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import time
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score
import numpy as np
from ast import literal_eval
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, AveragePooling2D
from keras.optimizers import SGD
from math import ceil
from keras.callbacks import EarlyStopping, Callback
from keras import backend as K
from itertools import tee  
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from keras.applications.vgg16 import preprocess_input, VGG16 
import tensorflow as tf
from collections import Counter
import matplotlib.pyplot as plt
from keras import layers, models, optimizers

Using TensorFlow backend.


## Costants

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH_ABSOLUTE = '/content/drive/My Drive/Laurea Magistrale/Adavanched Machine Learning/Progetto/Dataset/'

PATH_TRAIN = PATH_ABSOLUTE + 'Train/'
PATH_TEST = PATH_ABSOLUTE + 'Test/'

PATH_ANNOTATIONS_TRAIN =  PATH_TRAIN + 'Annotations'
PATH_IMAGES_TRAIN = PATH_TRAIN + 'Images'

PATH_MAIN_VAL = PATH_TRAIN + 'Main/val.txt'

PATH_ANNOTATIONS_TEST =  PATH_TEST + 'Annotations'
PATH_IMAGES_TEST = PATH_TEST + 'Images'

PATH = PATH_TRAIN 

PATH_MAIN = PATH + 'Main/'

PATH_IMAGES = PATH + 'Images/'

PATH_CSV_IMAGE_LABEL_TRAIN_VAL = PATH_ABSOLUTE + 'image_label_train_val.csv'
PATH_CSV_IMAGE_LABEL_TRAIN = PATH_ABSOLUTE + 'image_label_train.csv'
PATH_CSV_IMAGE_LABEL_VAL = PATH_ABSOLUTE + 'image_label_val.csv'
PATH_CSV_IMAGE_LABEL_TEST = PATH_ABSOLUTE + 'image_label_test.csv'

labels = ['filename', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
          'car', 'cat', 'chair', 'cow', 'diningtable',
          'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
          'train', 'tvmonitor']

## Creo dataframe da .txt in .../main (da fare solo prima volta)

In [0]:
train = pd.DataFrame(columns=labels)
val = pd.DataFrame(columns=labels)

# read all id of train and val
rows_train = []
f = open(PATH_MAIN + 'train.txt', "r")
for line in f.readlines():
  rows_train.append(line.split('\n')[0] + '.jpg')

rows_val = []
f = open(PATH_MAIN + 'val.txt', "r")
for line in f.readlines():
  rows_val.append(line.split('\n')[0] + '.jpg')

# initialize all row of dataframe to 0
train = train.append(pd.DataFrame(rows_train, columns=['filename']), ignore_index=True)
val = val.append(pd.DataFrame(rows_val, columns=['filename']), ignore_index=True)

for col in train.columns:
    if col != 'filename':
      train[col].values[:] = 0

for col in val.columns:
    if col != 'filename':
      val[col].values[:] = 0

# read txt in main and set 1 if there is that class in image
# if is -1 set to 0
for filename in os.listdir(PATH_MAIN):

    if filename.endswith("_train.txt"): 

      label = filename.split('_train.txt')[0]
      f = open(PATH_MAIN + filename, "r")

      for line in f.readlines():
        image = line.split(' ')[0] + '.jpg'
        presence = line.split(' ')[-1].strip()
        if presence == '1':
          train.loc[train['filename'] == image, label] = int(presence)

    elif filename.endswith("_val.txt"):

      label = filename.split('_val.txt')[0]
      f = open(PATH_MAIN + filename, "r")

      for line in f.readlines():
        image = line.split(' ')[0] + '.jpg'
        presence = line.split(' ')[-1].strip()
        if presence == '1':
          val.loc[val['filename'] == image, label] = int(presence)

train.to_csv(PATH_ABSOLUTE + 'train.csv')
val.to_csv(PATH_ABSOLUTE + 'val.csv')

## Lettura da csv

In [4]:
from sklearn.model_selection import train_test_split

train = pd.read_csv(PATH_ABSOLUTE + 'train.csv', index_col=0)
val = pd.read_csv(PATH_ABSOLUTE + 'val.csv', index_col=0)
'''
train_val = pd.concat([train, val])

x_train, x_val, y_train, y_val = train_test_split(  train_val['filename'], train_val[labels[1:]], 
                                                    test_size=0.2, random_state=42)
train_only = pd.concat([x_train, y_train], axis=1)
val_only = pd.concat([x_val, y_val], axis=1)
'''
# uso train come dato nel dataset
train_only = train

# divido validation in val e test
x_val, x_test, y_val, y_test = train_test_split(  val['filename'], val[labels[1:]], 
                                                  test_size=0.5, random_state=42)
val_only = pd.concat([x_val, y_val], axis=1)
test_only = pd.concat([x_test, y_test], axis=1)

print('Immagini di train: ', train_only.shape[0])
print('Immagini di validation: ', val_only.shape[0])
print('Immagini di test:', test_only.shape[0])

# calcolo frequenze label in train, test, validation
temp_train = train_only.drop(['filename'], axis=1)
temp_val = val_only.drop(['filename'], axis=1)
temp_test = test_only.drop(['filename'], axis=1)
freq_train = temp_train.apply(pd.value_counts).iloc[1,:]
freq_val = temp_val.apply(pd.value_counts).iloc[1,:]
freq_test = temp_test.apply(pd.value_counts).iloc[1,:]

print('Frequenze classi in train:')
print(freq_train)
print('Frequenze classi in val:')
print(freq_val)
print('Frequenze classi in test:')
print(freq_test)

Immagini di train:  5717
Immagini di validation:  2911
Immagini di test: 2912
Frequenze classi in train:
aeroplane       327
bicycle         268
bird            395
boat            260
bottle          365
bus             213
car             590
cat             539
chair           566
cow             151
diningtable     269
dog             632
horse           237
motorbike       265
person         1994
pottedplant     269
sheep           171
sofa            257
train           273
tvmonitor       290
Name: 1, dtype: int64
Frequenze classi in val:
aeroplane       178
bicycle         133
bird            190
boat            111
bottle          179
bus             106
car             280
cat             258
chair           286
cow              74
diningtable     139
dog             340
horse           128
motorbike       132
person         1063
pottedplant     148
sheep            81
sofa            123
train           147
tvmonitor       130
Name: 1, dtype: int64
Frequenze classi in test:


## Calcolo pesi delle diverse classi

In [0]:
temp = train_only.drop(['filename'], axis=1)

# frequency of labels
freq = temp.apply(pd.value_counts).iloc[1,:]

min_count = freq.min()

# between 0 and 1
freq_balance = freq.apply(lambda x: min_count/x)

## Image generator con data augmentation

In [6]:
from keras.applications.mobilenet import preprocess_input as preprocess_input_mobilenet
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

BATCH_SIZE = 64
IM_SIZE = (224, 224)

img_gen_train = ImageDataGenerator( rescale=1/255,
                                    preprocessing_function=preprocess_input_resnet50,
                                    #featurewise_center=True,
                                    #featurewise_std_normalization=True,
                                    #rotation_range=20,
                                    #width_shift_range=0.2,
                                    #height_shift_range=0.2,
                                    #horizontal_flip=True
                                  )
img_gen_val = ImageDataGenerator( rescale=1/255,
                                  preprocessing_function=preprocess_input_resnet50,
                                  #featurewise_center=True,
                                  #featurewise_std_normalization=True
                                )
#img_gen_train.mean = np.array([0.4589, 0.4355, 0.4032], dtype=np.float32).reshape((1,1,3)) 
#img_gen_train.std = np.array([0.2239, 0.2186, 0.2206], dtype=np.float32).reshape((1,1,3))
#img_gen_val.mean = np.array([0.4589, 0.4355, 0.4032], dtype=np.float32).reshape((1,1,3)) 
#img_gen_val.std = np.array([0.2239, 0.2186, 0.2206], dtype=np.float32).reshape((1,1,3))

train_generator = img_gen_train.flow_from_dataframe(
    train_only,
    shuffle=True,
    directory=Path(PATH_IMAGES),
    x_col='filename',
    y_col=labels[1:],
    class_mode='other',
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE,

)

val_generator = img_gen_val.flow_from_dataframe(
    val_only,
    shuffle=False,
    directory=Path(PATH_IMAGES),
    x_col='filename',
    y_col=labels[1:],
    class_mode='other',
    target_size=IM_SIZE,
    batch_size=BATCH_SIZE
)
TRAIN_LEN = train_generator.n
VAL_LEN = val_generator.n

Found 5715 validated image filenames.
Found 2911 validated image filenames.


## Metrica F1 e loss F1 on batch

In [0]:
def macro_soft_f1(y, y_hat):
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon())) 

def mAP(y_true, y_pred):

  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)

  y_true = tf.keras.backend.eval(y_true)
  y_pred = tf.keras.backend.eval(y_pred)

  _, classes = y_true.shape
      
  average_precisions = []

  for index in range(classes):
          row_indices_sorted = np.argsort(-y_pred[:, index])

          y_true_cls = y_true[row_indices_sorted, index]
          y_pred_cls = y_pred[row_indices_sorted, index]

          tp = (y_true_cls == 1)
          fp = (y_true_cls == 0)

          fp = np.cumsum(fp)
          tp = np.cumsum(tp)

          npos = np.sum(y_true_cls)

          rec = tp*1.0 / npos

          # avoid divide by zero in case the first detection matches a difficult
          # ground truth
          prec = tp*1.0 / np.maximum((tp + fp), np.finfo(np.float64).eps)

          mrec = np.concatenate(([0.], rec, [1.]))
          mpre = np.concatenate(([0.], prec, [0.]))

          # compute the precision envelope
          for i in range(mpre.size - 1, 0, -1):
              mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

          # to calculate area under PR curve, look for points
          # where X axis (recall) changes value
          i = np.where(mrec[1:] != mrec[:-1])[0]

          # and sum (\Delta recall) * prec
          average_precisions.append(np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1]))

  mAP = np.mean(average_precisions)
  
  return mAP

## Metrica F1 on epoch end

In [0]:
class Metrics(Callback):
  def __init__(self, validation_generator, validation_steps, threshold=0.5):
    self.validation_generator = validation_generator
    self.validation_steps = validation_steps
    self.threshold = threshold

  def on_train_begin(self, logs={}):
    self.val_f1_scores = []
    self.val_recalls = []
    self.val_precisions = []
    
  def on_epoch_end(self, epoch, logs={}):
    gen_1, gen_2 = tee(self.validation_generator)
    y_true = np.vstack(next(self.validation_generator)[1] for _ in range(self.validation_steps)).astype('int')
    y_pred = (self.model.predict_generator(self.validation_generator, steps=self.validation_steps) > self.threshold).astype('int')
    f1 = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    self.val_f1_scores.append(f1)
    self.val_recalls.append(recall)
    self.val_precisions.append(precision)
    print(f"- val_f1: {f1:.5f} - val_precision: {precision:.5f} - val_recall: {recall:.5f}")
    return

metrics = Metrics(val_generator, ceil(VAL_LEN/BATCH_SIZE))

## Modello con transfer learning da ResNet50

In [0]:
#!pip install extra_keras_metrics

from keras.applications import VGG16, ResNet50, VGG19
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import average_precision_score
#from extra_keras_metrics import 

base_model_resnet50 = ResNet50(weights = 'imagenet', input_shape=(224,224,3), include_top = False)

for layer in base_model_resnet50.layers:
    layer.trainable=False

x = base_model_resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(20, activation = 'sigmoid')(x)
model = Model(input = base_model_resnet50.input, output = predictions)

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam')

# callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

net_history = model.fit_generator(train_generator, epochs=50, verbose=1,
                  validation_data = val_generator,
                  steps_per_epoch = ceil(TRAIN_LEN/BATCH_SIZE),
                  validation_steps = ceil(VAL_LEN/BATCH_SIZE),
                  callbacks = [early_stop, metrics],
                  workers = 8,
                  use_multiprocessing=True,
                  class_weight = freq_balance
              )

## Plot epoche

In [0]:
import matplotlib.pyplot as plt
import matplotlib.style as style

def learning_curves(history):
    """Plot the learning curves of loss and macro f1 score 
    for the training and validation datasets.
    
    Args:
        history: history callback of fitting a tensorflow keras model 
    """
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    macro_f1 = history.history['macro_f1']
    val_macro_f1 = history.history['val_macro_f1']
    
    epochs = len(loss)

    style.use("bmh")
    plt.figure(figsize=(8, 8))

    plt.subplot(2, 1, 1)
    plt.plot(range(1, epochs+1), loss, label='Training Loss')
    plt.plot(range(1, epochs+1), val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')

    plt.subplot(2, 1, 2)
    plt.plot(range(1, epochs+1), macro_f1, label='Training Macro F1-score')
    plt.plot(range(1, epochs+1), val_macro_f1, label='Validation Macro F1-score')
    plt.legend(loc='lower right')
    plt.ylabel('Macro F1-score')
    plt.title('Training and Validation Macro F1-score')
    plt.xlabel('epoch')

    plt.show()

learning_curves(net_history)